In [1]:
# An implementation of logistics regression for binary class labels

In [2]:
# imports
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np 
from io import BytesIO

import torch
import torch.nn.functional as F

In [3]:
# preparing a toy dataset
#DATASET
ds = np.lib.DataSource()
fp = ds.open('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data')

x = np.genfromtxt(ByteIO(fp.read().encode()), delimeter=',', usecols=range(2), max_rows=100)
y = np.zeros(100)
y[50:] = 1

np.random.seed(1)
idx = np.arange(y.shape[0])
np.random.shuffle(idx)
X_test, y_test = x[idx[:25]], y[idx[:25]]
X_train , y_train = x[idx[25:]], y[idx[25:]]
mu, std = np.mean(X_train, axis=0), np.std(X_train, axis=0)
X_train, X_test = (X_train - mu)/ std, (X_test - mu)/ std

# make some plots
fig, ax = plt.subplots(1,2, figsize=(7, 2.5))
ax[0].scatter(X_train[y_train == 1, 0], X_train[y_train == 1, 1])
ax[0].scatter(X_train[y_train == 0,0], X_train[y_train == 0,1])
ax[1].scatter(X_test[y_test == 1,0], X_test[y_test == 1,1])
ax[1].scatter(X_test[y_test == 0,0], X_test[y_test == 0,1])
plt.show()

OSError: https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data not found.

In [4]:
# Low level implementation with manual gradients

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def custom_where(cond, x_1, x_2):
    return (cond * x_1) + ((1-cond) * x_2)

class LogisticRegression1():
    def __init__(self, num_features):
        self.num_features = num_features
        self.weights = torch.zeros(num_features, 1, 
                                  dtype=torch.float32, device=device)
        self.bias = torch.zeros(1, dtype=torch.float32, device=device)

    def forward(self, x):
        linear = torch.add(torch.mm(x, self.weights), self.bias)
        probas = self._sigmoid(linear)
        return probas
    
    def backward(self, probas, y):
        errors = y - probas.view(-1)
        return errors
    
    def predict_labels(self, x):
        probas = self.forward(x)
        labels = custom_where(probas >= .5, 1, 0)
        return labels
    
    def evaluate(self, x,y):
        labels = self.predict_labels(x).float()
        accuracy = torch.sum(labels.view(-1) == y) / y.size()[0]
        return accuracy
    
    def _sigmoid(self, z):
        return  1./(1 + torch.exp(-z))
    
    def logits_cost(self, y, proba):
        tmp1 = torch.mm(-y.view(1, -1), torch.log(proba))
        tmp2 = torch.mm(1 - y).view(1, -1), torch.log(1- proba)
        return tmp1 - tmp2
    
    def train(self, x, y, num_epochs, learning_rate=0.01):
        for e in range(num_epochs):
            
            ## compute your outputs ##
            probas = self.forward(x)
            
            ## compute the gradient ##
            errors = self.backward(probas, y)
            neg_grad = torch.mm(x.transpose(0,1), errors.view(-1, 1))
            
            ## update weights ##
            self.weights +=learning_rate * neg_gred
            self.bias += learning_rate * torch.sum(errors)
            
            ## Logging ##
            print('Epoch: '% (e+1), end="")
            print(' | Train ACC: %.3f' % self.evaluate(x,y), end="")
            print(' | Cost:%.3f' % self._logit_cost(y, self.forward(x)))

In [5]:
# tensors implementation
X_train_tensor = torch.tensor(X_train, dtype=torch.float32, device=device)
y_train_tensor = torch.tensor(y_train,dtype=torch.float32, device=device)

logr = LogisticRegression1(num_features=2)
logr.train((X_train_tensor, y_train_tensor, num_epochs, learning_rate=0.1))

print ('\n Model Parameters:')
print(' Weight: %s' % logr.bias)

SyntaxError: invalid syntax (<ipython-input-5-8ce57474ee08>, line 6)

In [6]:
# Evaluating the model
X_test_tensor = torch.tensor(X_test, dtypes=torch.float32, device=device)
y_test_tensor = torch.tensor(y_test, dtypes=torch.float32, device=device)

test_acc = logr.evaluate(X_test_tensor, y_test_tensor)
print('Test set Accuracy: %.2f%%' % (test_acc*100))

NameError: name 'X_test' is not defined

In [7]:
# 2D decision boundary

w, b = logr.weights, logr.bias 
x_min = -2
y_min = ( (-(w[0] * x_min) - b[0])
         /w[1])

x_max = 2
y_max = ( (-(w[0] * x_max) - b[0])
        / w[1] )

fig, ax = plt.subplots(1,2, sharex=True, figsize=(7,3))

ax[0].plot([x_min, x_max], [y_min, y_max])
ax[1].plot([x_min, x_max], [y_min, y_max])

ax[0].scatter(X_train[y_train==0,0], X_train[y_train==0,1], labels='class 0', marker='o')
ax[0].scatter(X_train[y_train==1,0], X_train[y_train==1,1], labels='class 1', marker='s')

ax[1].scatter(X_train[y_train==0,0], X_train[y_train==0,1], labels='class 0', marker='0')
ax[1].scatter(X_train[y_train==1,0], X_train[y_train==1,1], labels='class 1', marker='s')

ax[1].legend(loc='upper left')
plt.show()

NameError: name 'logr' is not defined

In [9]:
# Low level Implementation Using AutoGrad

def custom_where(cond, x_1, x_2):
    return (cond * x_1) + ((1-cond)* x_2)

class LogisticRegression2():
    def __init__(self, num_features):
        self.num_features = num_features
        
        self.weights = torch.zeros(num_features, 1,
                                  dtype=torch.float32,
                                  device=device,
                                  requires_grad=True) # autograd require.
        self.bias = torch.zeros(1,
                               dtype=torch.float32,
                               device=device,
                               requires_grad=True) # Grad required
        # forward propagation
        
        def forward(self, x):
            linear = torch.add(torch.mm(x, self.weights), self.bias)
            probas = self._sigmoid(linear)
            return probas
        
        def predict_labels(self, x):
            probas = self.forward(x)
            labels = custom_where((probas >= .5).float(), 1, 0)
            return labels
        
        def evaluate(self, x, y):
            labels = self.predict_labels(x)
            accuracy = (torch.sum(labels.view(-1) == y.view(-1))).float() / y.size()[0]
            return accuracy
        
        def _sigmoid(self, z):
            return 1. /(1 + torch.exp(-z))
        
        def _logit_cost(self, y, probas):
            tmp1 = torch.mm(-y.view(1, -1), torch.log(proba))
            tmp2 = torch.mm((1 - y).view(1, -1), torch.log(1- proba))
            return tmp1 - tmp2
        
        def train(self, x, y , num_epochs, learning_rate=0.01):
            
            for e in range(num_epochs):
                
                ## compute the outputs
                probas = self.forward(x)
                cost = self._logit_cost(y, proba)
                
                ## Compute gradients  ###
                cost.backward()
                
                ## updates the weights
                tmp = self.weights.detach()
                tmp -= learning_rate * self.weights.grad
                
                tmp = self.bias.detach()
                tmp -= learning_rate * self.bias.grad
                
                ## Reset gradients to zero for next iteration ###
                self.weights.grad.zero_()
                self.bias.grad.zero_()
                
                # Logging ###
                print('Epoch: %03d' % (e+1), end="")
                print(' | Train ACC: %.3f' % self.evaluate(x,y), end="")
                print(' | Cost: %.3f' % self._logit_cost(y, self.forward(x)))

In [11]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32, device=device)
y_train_tensor = torch.tensor(y_train, dtype= torch.float32, device=device)

logr = LogisticRegression2(num_features=2)
logr.train(X_train_tensor, y_train_tensor, num_epochs=10, learning_rate=0.1)

print('\n Model parameters:')
print('Weights %s' % logr.weights)
print('Bias %s' %logr.bias)

NameError: name 'X_train' is not defined

In [12]:
# Model Evaluation
X_test_tensor = torch.tensor(X_test, dtype=torch.float32, device=device)
y_test_tensor = toech.tensor(y_test, dtype=torch.float32,device=device)

test_acc = logr.evaluate(X_test_tensor, y_test_tensor)
print('Test set accuracy: %.2f%%' % (test_acc*100))

NameError: name 'X_test' is not defined

In [ ]:
## 2D decision boundary

w, b = logr.weights, logr.bias

x_min = -2
y_min = ( (-(w[0] * x_min) -b[0])
        /w[1])

x_max = 2
y_max = ( (-(w[0] * x_max) - b[0])
        / w[1])

fig, ax = plt.subplots(1,2, sharex=True, figsize=(7, 3))

ax[0].plot([x_min, x_max], [y_min, y_max])
ax[1].plot([x_min, x_max], [y_min, y_max])

ax[0].scatter(X_train[y_tran==0, 0], X_train[y_train==0, 1], labels='class 0' marker='o')
ax[0].scatter(X_train[y_train==1, 0], X_train[y_train==1,1], labels='class 1', marker='s')

ax[0].scatter(X_test[y_test==0, 0], X_test[y_test==0, 1], labels='class 0', marker='o')
ax[0].scatter(X_test[y_test==1, 0], X_test[y_test==1,1], labels='class 1')
